In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import MySQLdb

In [4]:
import warnings
warnings.simplefilter("ignore")

In [5]:
class Data:
    def __init__(self, table_name):
        self.__filename = table_name
        with open("D:/github_projects/login.txt") as file:
            self.__myserver_config = file.read().split(",") #config data for SQL server
            
        connect = MySQLdb.connect(*self.__myserver_config)
        cursor = connect.cursor()
        query = "SELECT * FROM titanium_pure"
        self.__data = pd.read_sql(query, connect)
        
        connect.close()


    
    @property
    def data(self):
        return self.__data


    @property
    def y(self):
        return self.__data["Density"]

    @property
    def X(self):
        return self.__data.drop("Density", axis=1)


    
    
    def __str__(self):
        return str(self.data)

    def __repr__(self):
        return str(self.data)

    
        

        
        
        
        
    

In [6]:
data1 = Data("titanium_pure")

In [7]:
data1

      id  Lazer  Power  Speed  Scanning_speed  Thickness Stratagy Density  \
0      1      1    160    700              80         50    Lines   95,71   
1      2      1    180    700              80         50    Lines   96,67   
2      3      1    200    700              80         50    Lines   97,00   
3      4      1    160    750              80         50    Lines   95,97   
4      5      1    180    750              80         50    Lines   96,48   
..   ...    ...    ...    ...             ...        ...      ...     ...   
487  488      1    370   1100             120         50   Chess4   96,73   
488  489      1    385   1100             120         50   Chess4   96,45   
489  490      1    395   1100             120         50   Chess4   97,81   
490  491      1    370   1100              90         50   Chess4   97,60   
491  492      1    370   1100             100         50   Chess4   97,04   

       Х    У Scanning_spot  
0     85   15           227  
1     60   15  